# CaLPA Scratch Folder

This is a scratch notebook used to test the code and functionality of the AI California Legislative Policy Analysis (CALPA) system. It is not intended for production use and may contain incomplete or experimental code. The purpose of this notebook is to facilitate the development and testing of the CALPA system, including its data processing, analysis, and visualization components. The notebook may include code snippets, comments, and notes related to the development process. Please refer to the official documentation and user guides for the CALPA system for more information on its usage and features.

In [ ]:
# %reset

## Initialization

In [ ]:
# Import required libraries
import os
import time
from datetime import date
from datetime import datetime
import json
import mimetypes
import glob
import base64
import zipfile
import io
import dotenv
import requests
import pandas as pd
import feedparser

In [2]:
# Load the Calpa module located in the scripts/python/calpa directory
from calpa import Calpa, LegiScan

# Load environment variables from .env file
dotenv.load_dotenv(os.path.join(os.getcwd(), '.env'))

# Instantiate the LegiScan and Calpa classes
calpa = Calpa()
legiscan = LegiScan()

# Create project metadata for the AI project
prjMetadata = calpa.projectMetadata("AI", "0")

# Create the project directories dictionary
prjDirs = calpa.projectDirectories(os.getcwd())

Project Global Settings:
- Name: California Legislative Policy Analysis
- Title: AI Legislative Policy Analysis
- Version: 1.0
- Author: Dr. Kostas Alexandridis, GISP
Data Dates
- Start Date: 2010-12-02
- End Date: 2025-04-19
- Periods: 2009-2010, 2011-2012, 2013-2014, 2015-2016, 2017-2018, 2019-2020, 2021-2022, 2023-2024, 2025-2026
Directory Global Settings:

General:
- Project: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA
- Admin: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\admin
- Metadata: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\metadata
- Analysis: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\analysis
Scripts:
- Python Calpa Module: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\calpa
- Markdown Scripts: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\markdown
- RIS Scripts: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\ris
Data:
- Main Data: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data
- Documents: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data\docs
- Le

## Session List

In [ ]:
# Get the list of sessions from LegiScan
sessionList = legiscan.getSessionList()

# Convert the sessionList to a pandas DataFrame
sessionDf = pd.DataFrame(sessionList)
sessionDf.head()

In [ ]:
# Obtain the stored sessions list from JSON dictionary on disk (data/lookup directory)
sessionListStored = legiscan.getStoredSessions()

In [ ]:
# Compare the sessionList and sessionListStored dictionaries for any changes
unmatchedSessions = legiscan.matchHash(sessionList, sessionListStored, "session_hash", silent=True)

# if the unmatchedSessions is empty, print "All sessions match", and delete the unmatchedSessions variable
if unmatchedSessions is None:
    print("All sessions match")
    del unmatchedSessions

In [ ]:
# Export the sessionList to a JSON file in the data/legiscan/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "sessionList.json"), "w", encoding="utf-8") as f:
    json.dump(sessionList, f, ensure_ascii=False, indent=4)

## Session People

In [ ]:
# Get the list of session people from LegiScan
sessionPeople = {}
for key, value in sessionList.items():
    sessionId = value["session_id"]
    sessionPeople[key] = legiscan.getSessionPeople(sessionId)

In [ ]:
# Obtain the stored session People list from JSON dictionary on disk (data/lookup directory)
sessionPeopleStored = legiscan.getStoredPeople()

In [ ]:
# Compare the sessionPeople and sessionPeopleStored dictionaries for any changes
# Create a dictionary to store unmatched people
unmatchedPeople = {}
# Iterate through each session and compare the people lists
for key, value in sessionPeople.items():
    unmatchedPeople[key] = {}
    unmatched = legiscan.matchHash(sessionPeople[key]["people"], sessionPeopleStored[key]["people"], "person_hash", silent=True)
    # If there are unmatched people, store them in the unmatchedPeople dictionary
    unmatchedPeople[key] = unmatched if unmatched is not None else None

# if the unmatchedPeople is empty, print "All people match", and delete the unmatchedPeople variable
if all(not value for value in unmatchedPeople.values()):
    print("All people match")
    # Delete the unmatchedPeople variable
    del unmatchedPeople

In [ ]:
# Export the sessionPeople to a JSON file in the data/legiscan/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "sessionPeople.json"), "w", encoding="utf-8") as f:
    json.dump(sessionPeople, f, ensure_ascii=False, indent=4)

## Dataset List

In [ ]:
# Get the list of datasets from LegiScan for each legislative session
datasetList = legiscan.getDatasetList()

In [ ]:
# Obtain the stored dataset list from JSON dictionary on disk (data/lookup directory)
datasetListStored = legiscan.getStoredDatasetList()

In [ ]:
# Compare the datasetList and datasetListStored dictionaries for any changes
unmatchedDatasets = legiscan.matchHash(datasetList, datasetListStored, "dataset_hash", silent=True)

# if the unmatchedSessions is empty, print "All sessions match", and delete the unmatchedSessions variable
if unmatchedDatasets is None:
    print("All datasets match")
    del unmatchedDatasets

In [ ]:
# export the datasetList to a JSON file in the data/legis/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "datasetList.json"), "w", encoding="utf-8") as f:
    json.dump(datasetList, f, ensure_ascii=False, indent=4)

## Misc

In [ ]:
aiBills = legiscan.getStoredBills("AI")
lcBills = legiscan.getStoredBills("LC")

In [ ]:
legiscan.updateStoredBills("AI", "XX999", 123555)

In [ ]:
legiscan.getStoredBills("AI")

In [ ]:
# Read the aiBills.json file from the data/lookup directory
aiBillsJson = os.path.join(prjDirs["pathDataLookup"], "aiBills.json")
with open(aiBillsJson, "r", encoding="utf-8") as f:
    aiBills = json.load(f)

In [ ]:
# Write the aiBills dictionay to a pandas DataFrame
aiBillsDf = pd.DataFrame(aiBills)
aiBillsDf.head()

In [5]:
ocealcrss = "https://legiscan.com/gaits/feed/eb2f0a37a652094577c62225fc31828c.rss"

In [7]:
ocealcfeed = feedparser.parse(ocealcrss)

In [8]:
ocealcfeed.feed.title

'LegiScan OCEA-LC Feed'

In [9]:
ocealcfeed.feed.link

'https://legiscan.com'

In [15]:
ocealcfeed["entries"][0]

{'title': 'CA AB1383',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://legiscan.com',
  'value': 'CA AB1383'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://legiscan.com/CA/bill/AB1383/2025?utm_campaign=rss&guid=2QgOZL6kAraEUIKr5vQLJ7'}],
 'link': 'https://legiscan.com/CA/bill/AB1383/2025?utm_campaign=rss&guid=2QgOZL6kAraEUIKr5vQLJ7',
 'summary': "An act to amend Sections 7522.10, 7522.25, and 7522.30 of, and to add Sections 7522.19 and 7522.26 to, the Government Code, relating to public employees’ retirement, and making an appropriation therefor.<br />\n<br />\n2025/04/11 From committee chair, with author's amendments: Amend, and re-refer to Com. on P. E. & R. Read second time and amended.",
 'summary_detail': {'type': 'text/html',
  'language': None,
  'base': 'https://legiscan.com',
  'value': "An act to amend Sections 7522.10, 7522.25, and 7522.30 of, and to add Sections 7522.19 and 7522.26 to, the Government Code, relati

In [21]:
ocealcfeed.keys()

dict_keys(['bozo', 'entries', 'feed', 'headers', 'etag', 'updated', 'updated_parsed', 'href', 'status', 'encoding', 'version', 'namespaces'])

In [34]:
ocealcfeed.entries[0]

{'title': 'CA AB1383',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://legiscan.com',
  'value': 'CA AB1383'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://legiscan.com/CA/bill/AB1383/2025?utm_campaign=rss&guid=2QgOZL6kAraEUIKr5vQLJ7'}],
 'link': 'https://legiscan.com/CA/bill/AB1383/2025?utm_campaign=rss&guid=2QgOZL6kAraEUIKr5vQLJ7',
 'summary': "An act to amend Sections 7522.10, 7522.25, and 7522.30 of, and to add Sections 7522.19 and 7522.26 to, the Government Code, relating to public employees’ retirement, and making an appropriation therefor.<br />\n<br />\n2025/04/11 From committee chair, with author's amendments: Amend, and re-refer to Com. on P. E. & R. Read second time and amended.",
 'summary_detail': {'type': 'text/html',
  'language': None,
  'base': 'https://legiscan.com',
  'value': "An act to amend Sections 7522.10, 7522.25, and 7522.30 of, and to add Sections 7522.19 and 7522.26 to, the Government Code, relati

In [ ]:
"https://api.legiscan.com/?key=APIKEY&op=getMonitorList&record=current"